## Imports

In [1]:
from pathlib import Path # reads paths in the current OS
import pandas as pd
import numpy as np
import yaml
import utils as ut
import plot_utils as pu
import matplotlib.pyplot as plt
from PIL import Image

## Reading

In [2]:
with open(Path('conf') / 'paths.yaml') as file:
    config = yaml.full_load(file)

data = pd.read_csv(Path(config['input_path']) / "all_data.csv", sep=",").fillna('')
locations_df = pd.read_csv(Path(config['input_path']) /  'municipis_merge.csv').fillna('')

In [3]:
data.shape

(540, 36)

In [4]:
vdp = pd.read_csv(Path(config['input_path']
                           ) / 'vdp_clean.csv').fillna('')

## Pre-process

In [5]:
data = data.merge(locations_df[['Comarca','Provincia']].drop_duplicates(), 
                  left_on = 'comarca_origin',right_on='Comarca',how='left')

In [6]:
data['comarca_origin'] = data['comarca_origin'].str.replace(r'^Urgell','Baix Urgell')

In [ ]:
# it takes a minute
com_coord = ut.get_comarca_coords(data)

In [ ]:
aux_text = {'total':'en total',
            'n_abastiment':'abastiment',
            'n_pagesos':'pagesos',
            'vegetables':'de verdures',
           'fruit':'de fruita',
           'meat':'de carn',
           'iseco':'ecos',
           'n_comarcas_delivery':'comarques amb enviament'}
max_size = 40

In [ ]:
n_columns = ['n_main_prod','n_other_prod','n_tot_prod','n_paym_methods','n_comarcas_delivery']

## Abastiment + pagesos producers in Venta de Proximitat dataset

In [ ]:
ds_intersection = data[data.marca_vdp != '']
ds_intersection.shape

In [ ]:
# plot n not percentages
cat = Image.open(Path(config['input_path'])
                 /'catalunya3.png')
x_low,x_up = 0.18,3.3
y_low,y_up = 40.5,42.9

## Comparing abastiment+pagesos and venta de proximitat

In [ ]:
n_columns_red = ['n_main_prod','n_other_prod','n_tot_prod']
aux_text_red = {'total':'en total',
            'vegetables':'de verdures',
           'fruit':'de fruita',
           'meat':'de carn'}
to_plot_vdp = ut.dataset_to_plot(vdp,vdp,com_coord,n_columns_red,multiple_origins=False)
to_plot = ut.dataset_to_plot(data,vdp,com_coord,n_columns,multiple_origins=True)

for col,txt in aux_text_red.items():
    #max_value = max(to_plot_vdp[col].max(),to_plot[col].max())
    fig = pu.plot_map_comarca_points(to_plot,cat,col,txt,max_size,to_plot[col].max(),
                                     x_low,x_up,y_low,y_up,txt.replace('en ','').replace('de ', '').title())
    pu.add_trace_plot(fig,to_plot_vdp,col,txt,max_size,to_plot_vdp[col].max(),
                      '#022d63',series_name='Venda de proximitat')
    name = config['input_path'] + "/out/" + "comparacio_vdp+covid_map_"+col+".html"
    fig.write_html(name)
    
    fig.show()

## Noves altes: Abastiment + Pagesos data compared to VdP

In [ ]:
#not in Venta de Proximitat
data_new    = data[data.marca_vdp == '']
to_plot = ut.dataset_to_plot(data_new,vdp,com_coord,n_columns,multiple_origins=True)

In [ ]:
title_name = 'Noves altes per comarca a venta de proximitat degut a la crisi del Covid-19'
fig = pu.create_figure()
fig = pu.add_trace_text_plot(fig,to_plot,'pctge_new','nous',max_size,to_plot['pctge_new'].max(),'#63022d')
fig = pu.plot_layout(fig,cat,x_low,x_up,y_low,y_up,title_name)
fig.write_image("noves_altes_comarca_mapa.png")
fig.write_image("noves_altes_comarca_mapa.pdf")
fig.show()

In [ ]:
title_name = 'Noves altes per comarca a venta de proximitat degut a la crisi del Covid-19'
fig = pu.create_figure()
fig = pu.add_trace_text_plot(fig,to_plot,'total','nous',max_size,to_plot['total'].max(),'#63022d')
fig = pu.plot_layout(fig,cat,x_low,x_up,y_low,y_up,title_name)
fig.write_image("num_noves_altes_comarca_mapa.png")
fig.write_image("num_noves_altes_comarca_mapa.pdf")
fig.show()

## Abastiment + Pagesos data

In [ ]:
to_plot = ut.dataset_to_plot(data,vdp,com_coord,n_columns,multiple_origins=True)

In [ ]:
# plot n not percentages
cat = Image.open(Path(config['input_path']) /'catalunya3.png')
x_low,x_up = 0.18,3.3
y_low,y_up = 40.5,42.9


for col,txt in aux_text.items():
    fig = pu.plot_map_comarca_points(to_plot,cat,col,txt,max_size,to_plot[col].max(),
                                     x_low,x_up,y_low,y_up,txt.replace('en ','').replace('de ', '').title())
    name = config['input_path'] + "/out/" + "abastiment+pagesos_map_"+col+".html"
    fig.write_html(name)
    fig.show()

## Plots separating dataset origin

### Related to columns with counts

In [ ]:
aux_text = {'n_tot_prod':'Mitja productes totals',
           'n_other_prod':'Mitja productes no principals',
           'n_main_prod':'Mitja productes principals',
            'n_paym_methods':'Mitja métodes de pagament'}
for col,txt in aux_text.items():
    fig = pu.bar_perc_separate_datasets(data, col, txt)
    name = config['input_path'] + "/out/" + "abastiment+pagesos_bar_"+col+".html"
    fig.write_html(name)
    fig.show()

In [ ]:
pagament = {
    'paym_card':'Targeta',
    'paym_cash':'Efectiu',
    'paym_bizum':'Bizum',
    'paym_trans':'Tranferència',
    'card+cash': 'Efectiu + targeta',
    'trans+cash': 'Efectiu + tranferència',
    'card+trans': 'Transferència + targeta',
    'card+bizum': 'Targeta + Bizum',
    'cash+bizum': 'Efectiu + Bizum',
    'trans+bizum':'Tranferència + Bizum'
}

In [ ]:
data = ut.add_payment_types(data,pagament)

In [ ]:
ab_gb,pag_gb = ut.pagament_prep(data,pagament)
fig = pu.bar_payment_type(pag_gb,ab_gb)
fig.show()

### Related to other products with some filters

In [ ]:
'''pu.bar_perc_separate_datasets(data[(data.n_main_prod==0) & (data.n_tot_prod>0)],
                         'n_other_prod',aux_text['n_other_prod'])
pu.bar_perc_separate_datasets(data[(data.n_main_prod==1)],'n_other_prod',aux_text['n_other_prod'])
pu.bar_perc_separate_datasets(data[(data.n_main_prod>=2)],'n_other_prod',aux_text['n_other_prod'])'''

Quan no venen productes principals, la majoria venen únicament un producte (no principal).

Quan venen un producte principal, la majoria no venen altre tipus de producte.

Quan venen més d'un producte principal, acostumen a vendre més tipus de productes.